In [1]:
#@title Importaciones

import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#@title importacion de archivos y formateo de columnas

anime_dataset = pd.read_csv("./anime.csv", sep = '\t',on_bad_lines='skip')

anime_dataset.columns = anime_dataset.columns.str.lower()
anime_dataset.columns = anime_dataset.columns.str.replace(' ', '_')

In [4]:
anime_dataset = anime_dataset[anime_dataset['anime_id'] != 44303] # anime que cumple con los 200 caracteres en synopsis pero no contiene datos utiles para su estudio
anime_dataset_filt = pd.DataFrame(anime_dataset[ anime_dataset["synopsis"].str.len() >= 200 ])
anime_dataset_filt = anime_dataset_filt.sort_values(by='popularity_rank')
anime_dataset_filt.index = range(0,len(anime_dataset_filt)) 

In [10]:
#@title Eliminando columnas

col_delete_list = ["score_count", "score_rank","type", "status", "num_episodes", "start_date",
                   "end_date", "season", "studios", "clubs", "score_01_count",
                   "score_02_count", "score_03_count", "score_04_count", "score_05_count",
                   "score_06_count", "score_07_count", "score_08_count", "score_09_count",
                   "score_10_count", "num_episodes", "watching_count",
                   "completed_count","on_hold_count","dropped_count"]

filter_columns = list(filter(lambda x: True if x not in col_delete_list else False, list(anime_dataset_filt.columns)))

anime_dataset_filt = anime_dataset_filt[filter_columns]


anime_dataset_filt['longitud_synopsis_caracteres'] = anime_dataset_filt['synopsis'].apply(len) 
anime_dataset_filt['genres'] = anime_dataset_filt['genres'].apply(lambda x: x.split('|'))

In [ ]:
data_generos = pd.DataFrame(list(set(elemento for sublista in anime_dataset_filt['genres'] for elemento in sublista)), columns=['genres_name'])

anime_genres_relation = anime_dataset_filt.explode('genres')[["anime_id", "genres"]] # expande anime_dataset_filt para dividir las listas en filas individuales

data_generos = anime_genres_relation.groupby('genres').size().reset_index(name='cantidad') # Agrupamos por el género y contamos la cantidad de animes en cada uno
data_generos.columns = ['genres_name', 'cantidad'] 

anime_genres_relation

,anime_id,genres
0,16498,Action
0,16498,Drama
0,16498,Fantasy
0,16498,Mystery
0,16498,Military
...,...,...
8504,50710,Comedy
8504,50710,Romance
8504,50710,Sci-Fi
8504,50710,Shounen


In [ ]:
#@title Analisis de correlacion entre longitud de sinopsis y la puntuacion
correlacion_longSynop_score = anime_dataset_filt[['longitud_synopsis_caracteres', 'score']].corr()


# plt.scatter(anime_dataset_filt['longitud_synopsis_caracteres'], anime_dataset_filt['score'], color='blue')
# plt.title('Relación entre Longitud de la Sinopsis y Rango de Popularidad')
# plt.xlabel('Longitud de la Sinopsis')
# plt.ylabel('La puntuación')
# plt.show()
anime_dataset_filt[['longitud_synopsis_caracteres', 'score']]
anime_dataset_filt.groupby('score').agg({'score':'count'})
correlacion_longSynop_score
# anime_dataset_filt[["genres", "anime_id"]]

,genres,genres_ID,cantidad
0,0,0,6722
1,1,1,2351
2,2,2,5804
3,3,3,2205
4,4,4,2025
5,5,5,4351
6,6,6,16702
7,7,7,540
8,8,8,5
9,9,9,3533
